# Load and test pretrained model:
### There is no test partition, create a valid partition randomly and test the pretrained model's accuracy

In [33]:
experiment_name= "KaggleFashion_pretrain_classification_resnet34"

In [17]:
import glob
import cv2

import matplotlib.pyplot as plt
%matplotlib inline
from pandas.core.common import flatten
import numpy as np
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

from datasets import *
import networks
import utils


import ast
import PIL


In [18]:
#######################################################
#                  Create Dataset
#######################################################
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

#######################################################
#                  Create Dataset
#######################################################
# dataset_train = datasets.FashionProductsDataset(img_dir="./images",train_path="DeepFashionDataset_train.csv",mode="train", transform= transform)
dataset_train = FashionProductsDataset(img_dir="./Kaggle_fashion/images", styles_dir="./Kaggle_fashion/styles.csv")

/tmp/ipykernel_4752/161129884.py:16: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  dataset_train = FashionProductsDataset(img_dir="./Kaggle_fashion/images", styles_dir="./Kaggle_fashion/styles.csv")
/tmp/ipykernel_4752/161129884.py:16: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  dataset_train = FashionProductsDataset(img_dir="./Kaggle_fashion/images", styles_dir="./Kaggle_fashion/styles.csv")


In [19]:
len(dataset_train)

44419

In [20]:
len(dataset_train.classes_set)

47

### # Create valid data set:

In [21]:
from torch.utils.data.sampler import SubsetRandomSampler
# percentage of training set to use as validation
valid_size = 0.3

# obtain training indices that will be used for validation
num_train= len(dataset_train)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [22]:
# check if CUDA is available
cuda = torch.cuda.is_available()

if not cuda:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [23]:
batch_size= 128
num_workers = 0 # number of subprocesses to use for data loading

# prepare data loaders:
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)

# Create the same model:

In [27]:
# Load the pretrained model from pytorch
embedding_net= torchvision.models.resnet34(pretrained=True)

# print out the model structure
print(embedding_net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [28]:
embedding_net.fc= nn.Sequential(nn.Linear(512, 128),
                                nn.BatchNorm1d(128),
                                nn.ReLU(),
                                nn.Linear(128, 47)
                                )

In [29]:
# Freeze training for all layers parameters:
for param in embedding_net.parameters():
    param.required_grad = False    

In [30]:
model= embedding_net

In [31]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Load the trained model:

In [34]:
PATH= experiment_name+'_100epoch.pt'

check_point= torch.load(PATH)
model.load_state_dict(check_point['model_state_dict'])
epoch = check_point['epoch']
loss_val = check_point['loss']

optimizer.load_state_dict(check_point['optimizer_state_dict'])

In [36]:
device = torch.device('cuda')
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [46]:
# initialize lists to monitor test loss and accuracy
nbr_class= 47
test_loss = 0.0
class_correct = list(0. for i in range(nbr_class))
class_total = list(0. for i in range(nbr_class))

model.eval() # prep model for evaluation

for data, target in valid_loader:
    # move tensors to GPU if CUDA is available
    data= data.float() # cast to float
    if cuda:
        data, target= data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = loss_fn(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(len(data)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(valid_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(nbr_class):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(valid_loader.dataset.classes_set[i]), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % str(valid_loader.dataset.classes_set[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.021701

Test Accuracy of Apparel/Topwear: 99% (4526/4540)
Test Accuracy of Apparel/Bottomwear: 99% (789/793)
Test Accuracy of Accessories/Watches: 99% (792/795)
Test Accuracy of Accessories/Socks: 98% (192/195)
Test Accuracy of Footwear/Shoes: 99% (2190/2199)
Test Accuracy of Accessories/Belts: 100% (244/244)
Test Accuracy of Footwear/Flip Flops: 97% (258/264)
Test Accuracy of Accessories/Bags: 98% (936/946)
Test Accuracy of Apparel/Innerwear: 98% (550/560)
Test Accuracy of Footwear/Sandal: 97% (277/284)
Test Accuracy of Accessories/Shoe Accessories: 75% ( 6/ 8)
Test Accuracy of Personal Care/Fragrance: 99% (314/317)
Test Accuracy of Accessories/Jewellery: 100% (335/335)
Test Accuracy of Personal Care/Lips: 98% (151/153)
Test Accuracy of Apparel/Saree: 99% (138/139)
Test Accuracy of Accessories/Eyewear: 99% (320/322)
Test Accuracy of Personal Care/Nails: 100% (104/104)
Test Accuracy of Accessories/Scarves: 82% (34/41)
Test Accuracy of Apparel/Dress: 91% (123/135)
Test Accu